# ETL 2. Transformación 1 - Limpieza

In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast 

from datetime import datetime, timedelta

pd.options.display.max_columns=None

## Introducción:
Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

- El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
- El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

**El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes**. 

Para ello:
1. Cargaremos los dos ficheros de datos del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea  
  

In [2]:
clima = pd.read_csv('datos/ETL-1.csv', index_col = 0)
clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 40, 'speed': ...",12,15,2,1028,none,0,0,315,2,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 325, 'speed':...",14,15,1,1027,none,0,0,50,2,USA


In [3]:
df = pd.read_csv('datos/attacks_limpieza_completa.csv', index_col=0)
df.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


### Nos quedamos con los países que nos piden:

In [4]:
list_country = ['usa', 'australia', 'south africa', 'new zealand', 'papua new guinea']

In [5]:
df_country = df[df['country'].isin(list_country)]
df_country.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925
5,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925


In [6]:
df_country['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

### Desempaquetamos la columna 'hr_profile':

In [7]:
clima['rh_profile'] = clima['rh_profile'].apply(ast.literal_eval)

In [8]:
df_rh = clima['rh_profile'].apply(pd.Series)

In [9]:
df_rh.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 1}","{'layer': '900mb', 'rh': 1}","{'layer': '850mb', 'rh': 4}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 6}","{'layer': '700mb', 'rh': 6}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 8}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 1}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': 1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 6}","{'layer': '200mb', 'rh': 6}"
1,"{'layer': '950mb', 'rh': 1}","{'layer': '900mb', 'rh': 2}","{'layer': '850mb', 'rh': 5}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 4}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 5}","{'layer': '550mb', 'rh': 6}","{'layer': '500mb', 'rh': 5}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': 3}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 0}","{'layer': '200mb', 'rh': 3}"


In [10]:
for i in range(len(df_rh.columns)):
    nombre_rh = "rh_"+ str(df_rh[i].apply(pd.Series)["layer"][0])
    valores_rh = (df_rh[i].apply(pd.Series)["rh"])

    clima.insert(i, nombre_rh, valores_rh)

In [11]:
clima.head()

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,1,1,4,6,6,6,7,8,5,3,1,-1,1,2,6,6,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 40, 'speed': ...",12,15,2,1028,none,0,0,315,2,USA
1,1,2,5,6,4,3,4,5,6,5,2,-1,3,5,0,3,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 325, 'speed':...",14,15,1,1027,none,0,0,50,2,USA
2,1,4,5,5,2,2,2,3,4,5,6,4,3,1,1,5,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 190, 'speed':...",14,15,3,1026,none,0,0,85,2,USA
3,3,4,5,4,2,1,1,2,1,2,4,6,5,5,4,3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 235, 'speed':...",13,10,8,1026,none,0,0,30,2,USA
4,4,4,4,3,2,1,0,0,0,0,4,4,11,12,9,6,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 215, 'speed':...",13,15,8,1027,none,0,0,5,2,USA


### Desempaquetamos la columna 'wind_profile':

In [12]:
clima['wind_profile'] = clima['wind_profile'].apply(ast.literal_eval)

In [13]:
df_wind = clima['wind_profile'].apply(pd.Series)
df_wind.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 40, 'speed': 1}","{'layer': '900mb', 'direction': 5, 'speed': 3}","{'layer': '850mb', 'direction': 335, 'speed': 3}","{'layer': '800mb', 'direction': 335, 'speed': 4}","{'layer': '750mb', 'direction': 325, 'speed': 4}","{'layer': '700mb', 'direction': 325, 'speed': 4}","{'layer': '650mb', 'direction': 330, 'speed': 5}","{'layer': '600mb', 'direction': 325, 'speed': 5}","{'layer': '550mb', 'direction': 315, 'speed': 5}","{'layer': '500mb', 'direction': 320, 'speed': 6}","{'layer': '450mb', 'direction': 325, 'speed': 6}","{'layer': '400mb', 'direction': 320, 'speed': 6}","{'layer': '350mb', 'direction': 325, 'speed': 6}","{'layer': '300mb', 'direction': 330, 'speed': 6}","{'layer': '250mb', 'direction': 330, 'speed': 7}","{'layer': '200mb', 'direction': 345, 'speed': 7}"
1,"{'layer': '950mb', 'direction': 325, 'speed': 2}","{'layer': '900mb', 'direction': 310, 'speed': 2}","{'layer': '850mb', 'direction': 305, 'speed': 3}","{'layer': '800mb', 'direction': 320, 'speed': 4}","{'layer': '750mb', 'direction': 320, 'speed': 4}","{'layer': '700mb', 'direction': 325, 'speed': 4}","{'layer': '650mb', 'direction': 330, 'speed': 5}","{'layer': '600mb', 'direction': 330, 'speed': 5}","{'layer': '550mb', 'direction': 325, 'speed': 5}","{'layer': '500mb', 'direction': 325, 'speed': 5}","{'layer': '450mb', 'direction': 320, 'speed': 5}","{'layer': '400mb', 'direction': 315, 'speed': 5}","{'layer': '350mb', 'direction': 305, 'speed': 5}","{'layer': '300mb', 'direction': 305, 'speed': 6}","{'layer': '250mb', 'direction': 320, 'speed': 6}","{'layer': '200mb', 'direction': 330, 'speed': 6}"


In [14]:
for i in range(len(df_wind.columns)): 

    nombre = "wind_direction_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 

    valores = list(df_wind[i].apply(pd.Series)["direction"])

    clima.insert(i, nombre, valores)

In [15]:
clima.head()

,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,40,5,335,335,325,325,330,325,315,320,325,320,325,330,330,345,1,1,4,6,6,6,7,8,5,3,1,-1,1,2,6,6,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 40, 'speed': ...",12,15,2,1028,none,0,0,315,2,USA
1,325,310,305,320,320,325,330,330,325,325,320,315,305,305,320,330,1,2,5,6,4,3,4,5,6,5,2,-1,3,5,0,3,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 325, 'speed':...",14,15,1,1027,none,0,0,50,2,USA
2,190,270,295,310,310,315,315,315,325,325,325,320,315,305,300,315,1,4,5,5,2,2,2,3,4,5,6,4,3,1,1,5,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 190, 'speed':...",14,15,3,1026,none,0,0,85,2,USA
3,235,250,285,305,305,310,315,315,310,315,310,305,295,300,315,325,3,4,5,4,2,1,1,2,1,2,4,6,5,5,4,3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 235, 'speed':...",13,10,8,1026,none,0,0,30,2,USA
4,215,250,275,290,295,295,295,300,305,305,310,305,310,305,305,310,4,4,4,3,2,1,0,0,0,0,4,4,11,12,9,6,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 215, 'speed':...",13,15,8,1027,none,0,0,5,2,USA


In [16]:
for i in range(len(df_wind.columns)): 

    nombre = "wind_speed_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 

    valores = list(df_wind[i].apply(pd.Series)["speed"])

    clima.insert(i, nombre, valores)

In [17]:
clima.head()

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,1,3,3,4,4,4,5,5,5,6,6,6,6,6,7,7,40,5,335,335,325,325,330,325,315,320,325,320,325,330,330,345,1,1,4,6,6,6,7,8,5,3,1,-1,1,2,6,6,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 40, 'speed': ...",12,15,2,1028,none,0,0,315,2,USA
1,2,2,3,4,4,4,5,5,5,5,5,5,5,6,6,6,325,310,305,320,320,325,330,330,325,325,320,315,305,305,320,330,1,2,5,6,4,3,4,5,6,5,2,-1,3,5,0,3,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 325, 'speed':...",14,15,1,1027,none,0,0,50,2,USA
2,2,3,3,4,4,4,4,5,5,5,5,5,5,5,5,6,190,270,295,310,310,315,315,315,325,325,325,320,315,305,300,315,1,4,5,5,2,2,2,3,4,5,6,4,3,1,1,5,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 190, 'speed':...",14,15,3,1026,none,0,0,85,2,USA
3,2,3,3,4,4,4,4,4,5,5,5,5,5,5,5,5,235,250,285,305,305,310,315,315,310,315,310,305,295,300,315,325,3,4,5,4,2,1,1,2,1,2,4,6,5,5,4,3,12,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 235, 'speed':...",13,10,8,1026,none,0,0,30,2,USA
4,2,3,3,3,3,4,4,4,5,5,5,5,5,5,5,5,215,250,275,290,295,295,295,300,305,305,310,305,310,305,305,310,4,4,4,3,2,1,0,0,0,0,4,4,11,12,9,6,15,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 215, 'speed':...",13,15,8,1027,none,0,0,5,2,USA



4. Guardad los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.